Inspired by [this question](https://math.stackexchange.com/questions/4352292/solving-a-parabolic-pde-numerically-with-the-spectral-method) Ali developed a Python Script for solving the following PDE on $x \in [-3, 3]$ and $t \in [0, 0.5]$ using the Spectral Method:

$$
\frac{\partial p}{\partial t} = (12x^2-4) p + \left[4x(x^2-1)+0.1\right] \frac{\partial p}{\partial x} + \frac{1}{\beta} \frac{\partial^2 p}{\partial x^2} \\
$$

The initial condition is $p(x,0)=\frac{1}{\sqrt{2\pi}} \exp(-\frac{x^2}{2})$ and the zero boundary condition is $p(-3,t)=p(3,t)=0$.

In [1]:
# Script is developed by Ali (at MSE):
# https://math.stackexchange.com/a/4352909/993738
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

def weights(z, x, m):
    n = len(x) - 1
    c1 = 1.
    c4 = x[0] - z
    c = np.zeros((n + 1, m + 1), dtype=float)
    c[0, 0] = 1.
    for i in range(n + 1):
        mn = min(i, m)
        c2 = 1.
        c5 = c4
        c4 = x[i] - z
        for j in range(i):
            c3 = x[i] - x[j]
            c2 = c2 * c3
            if j == i - 1:
                for k in range(mn, 0, -1):
                    c[i, k] = c1 * (k * c[i - 1, k - 1] - c5 * c[i - 1, k]) / c2
                c[i, 0] = -c1 * c5 * c[i - 1, 0] / c2
            for k in range(mn, 0, -1):
                c[j, k] = (c4 * c[j, k] - k * c[j, k - 1]) / c3
            c[j, 0] = c4 * c[j, 0] / c3
        c1 = c2
    return c

T = 0.5
n = 20  # The number of time steps.
dt = T / n

N = 50  # The number of collocation points in the spatial dimension.
beta = 1

x = 3 * np.cos(np.linspace(-np.pi, 0, N))
w = [weights(xi, x, 2) for xi in x]
D = np.array([c[:, 1] for c in w]) / 3

A = np.eye(N) - D @ (np.diag(4 * x * (x ** 2 - 1) + 0.1) + (1 / beta) * D) * dt
A = A[1:N-1, 1:N-1]  # That the boundary conditions are zero makes things simple.

def step(p):
    b = p[1:N-1]
    return np.hstack([0, np.linalg.solve(A, b), 0])

# Grid Vars
x_min = -3
x_max = 3

y_min = 0
y_max = 0.5

x_grid = np.linspace(x_min, x_max, 100)
w = np.array([weights(gi, x, 0) for gi in x_grid]).reshape(100, N)
z = []

p = np.exp(-(x ** 2) / 2) / np.sqrt(2 * np.pi)
z.append(w @ p)

for i in range(n):
    p = step(p)
    z.append(w @ p)

y_grid = np.linspace(y_min, y_max, n + 1)

fig = go.Figure(data=[go.Surface(
    contours = {
        "x": {"show": True, "color":"white"},
        "y": {"show": True, "color":"white"},
        "z": {"show": True, "color":"white"}
    },
    x=x_grid,
    y=y_grid,
    z=z,
    colorscale='Viridis',
    showscale=False,
    lighting=dict(roughness=0.9))])
fig.update_layout(title='Spectral Method', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.update_traces(contours_z=dict(show=True, usecolormap=True, highlightcolor="limegreen", project_z=True))
fig.show()

#fig, ax = plt.subplots()
#ax.plot(grid, w @ p)